## Structure of Our Milvus Vector Index

The structure of Vector Index is as follows:
* The collection is named `Chunk_Vector_Index`
* It has a default partition named `_default` created automatically at the time of creation
* For every pdf file uploaded, we create a new partition with a unique, randomly chosen name. refer to `modules.py` for code
* the chunks for that pdf file are uploaded in its own unique partition



Note that a particular partition is dependent on following things:
1. Schema:
    + Our Schema right now is:
        * ID(`id`)
        * chunk_vector(`vector`)
        * chunk_text(`text`)
        * source document(`source`)
        * page number of the chunk in document(`page`)
2. Embedding dimension: embedding dimension will be input at the time of creating the collection. So incase we are choosing a different embedding model, we will need to create a new collection. For now the embedding model is opensource `HuggingFaceEmbeddings()`

In [4]:
# %pip install langchain_huggingface
%pip install --quiet sentence-transformers pymilvus langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 13.8 MB/s eta 0:00:00


In [6]:
!pip install grpcio>=1.49.1

In [8]:
!pip install --force-reinstall "grpcio<=1.63.0,>=1.49.1"

  Using cached grpcio-1.63.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.6 MB)
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.49.1
    Uninstalling grpcio-1.49.1:
      Successfully uninstalled grpcio-1.49.1


In [1]:
from pymilvus import MilvusClient, connections,Collection
from langchain_huggingface import HuggingFaceEmbeddings

MILVUS_URI = "http://4.213.193.212:19530"

milvus_client = MilvusClient(MILVUS_URI)


DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 80c0ad1b7dce468780de2445f9b7317b


In [2]:
embedding_model = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
def embed_text(text):
    return embedding_model.embed_query(text)

In [4]:
test_embedding = embed_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)

768


In [6]:
res = milvus_client.list_collections()

res

[]

In [7]:
collection_name = "Chunk_Vector_Index"

In [8]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [9]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="COSINE",  # Cosine similarity
    consistency_level="Strong",  # Strong consistency level
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: Chunk_Vector_Index
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: Chunk_Vector_Index


In [10]:
res = milvus_client.list_collections()

res

['Chunk_Vector_Index']

In [16]:
# milvus_client.create_partition(collection_name,partition_name='owais')
# milvus_client.release_partitions(collection_name,partition_names=['owais'])
# milvus_client.drop_partition(collection_name,partition_name='owais')
# milvus_client.list_partitions(collection_name)